In [ ]:
import os
import plotly.graph_objs as go
from plotly.offline import iplot
from pathlib import Path
import git
import numpy as np

In [ ]:
# set autoreload to reload all external modules automatically (otherwise changes to those modules won't take effect in the notebook)
%load_ext autoreload
%autoreload 2
from make_folder_dataset import MakeFolderDataset

In [ ]:
def get_git_root():
    git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
    return git_repo.git.rev_parse("--show-toplevel")

In [ ]:
# load dataset instances
# use current git repo root folder as a reliable "base" folder. Dataset folders should be under <git-root>\dataset\
instances = []
dataset_path = Path(get_git_root()) / "dataset"
for p in dataset_path.iterdir():
    if p.is_dir():
        instance = MakeFolderDataset(p.absolute())
        instance.extract_robot_data()
        instance.get_labels_all()
        instances.append(instance)

In [ ]:
# add labels to robot-data dataframe for all instances
for inst in instances:
    inst.df['has_contact'] = 0
    inst.df['contact_type'] = 0

    inst.true_label = inst.true_label.drop(['DATA1', 'DATA2', 'DATA3' ,'DATA4'], axis=1)

    # find 1st row in robot-data that corresponds (time-wise) to 1st row in true_label
    filtered_df = inst.df[0:][inst.df['time'] - inst.true_label.iloc[0]['time'] > 0]
    first_row_index = inst.df.index.get_loc(filtered_df.index[0])

    j = 0
    for i in range(first_row_index, len(inst.df)):
        # find next higher (time) true_label row or break if none exist
        filtered_inst_true_label = inst.true_label[inst.true_label['time'] - inst.df.iloc[i]['time'] > 0]
        if len(filtered_inst_true_label) == 0:
            break

        j = inst.true_label.index.get_loc(filtered_inst_true_label.index[0])

        # if absolute diff between robot-data and next higher label time is below threshold (0.02), set has_contact label to 1
        if abs(inst.df.loc[i, 'time'] - inst.true_label.loc[j, 'time']) < 0.02:
            inst.df.loc[i, 'has_contact'] = 1

In [ ]:
# clean up faulty (true_label) sensor data
# IMPORTANT: assume that the first measurement (time-window with has_contact = 1) is correct and can be used as a reference point to clean the remaining instance
# thus it must manually be verified that the first measurement of each instance is indeed correct

def get_contact_duration(df, time):
    start_time_index = df[(df['time'] < time) & (
        df['has_contact'] == 0)].index[-1] + 1
    start_time = df.loc[start_time_index, 'time']
    end_time_index = df[(inst.df['time'] > time) & (
        df['has_contact'] == 0)].index[0] - 1
    end_time = df.loc[end_time_index, 'time']
    return (end_time - start_time), start_time, end_time


def get_next_contact_time(df, excl_from_time):
    filtered_df = df[(df['time'] > excl_from_time) & (df['has_contact'] == 1)]
    return (filtered_df.iloc[0]['time'], filtered_df.index[0]) if len(filtered_df) > 0 else (None, None)


for inst in instances:
    # calculate duration of 1st contact time-window
    contact_time, _ = get_next_contact_time(inst.df, -1)
    reference_duration, reference_start_time, reference_end_time = get_contact_duration(
        inst.df, contact_time)

    inst.first_contact_start_time = reference_start_time
    inst.window_size = window_size = len(inst.df[(inst.df['time'] >= reference_start_time) & (
        inst.df['time'] <= reference_end_time)])

    # set has_contact to 0 for all contact time-windows with duration outside of [0.95*reference_duration, 1.05*reference_duration] -> remove faulty contact time-windows
    # (as later only data during contact (has_contact = 1) will be used)
    last_contact_end_time = reference_end_time
    while True:
        contact_time, _ = get_next_contact_time(inst.df, last_contact_end_time)
        if contact_time is None:
            break
        contact_duration, contact_start_time, contact_end_time = get_contact_duration(
            inst.df, contact_time)
        if not (0.95 * reference_duration <= contact_duration <= 1.05 * reference_duration):
            # print(contact_duration, contact_start_time, contact_end_time)
            inst.df.loc[(inst.df['time'] >= contact_start_time) & (
                inst.df['time'] <= contact_end_time), 'has_contact'] = 0
        last_contact_end_time = contact_end_time

In [ ]:
# target = ['e0','e1','e2','e3','e4','e5','e6']
# target = ['de0','de1','de2','de3','de4','de5','de6']
target = ['etau_J0','etau_J1', 'etau_J2', 'etau_J3', 'etau_J4', 'etau_J5', 'etau_J6']

#for i in target:
for i in target[0:1]:
    for inst in instances:
        # label gets scaled otherwise measure and label are not visible properly on plot
        A = inst.df[i].max()-inst.df[i].min()
        inst.df['has_contact_scaled'] = inst.df['has_contact'] * \
            A + inst.df[i].min()

        # use plotly to make interactive plots
        trace1 = go.Scatter(
            x=inst.df['time'], y=inst.df['has_contact_scaled'], name='has contact')
        trace2 = go.Scatter(
            x=inst.df['time'], y=inst.df[i], mode='lines', name='robot data')
        data = [trace1, trace2]
        layout = go.Layout(title=f'{i} (instance {os.path.basename(os.path.normpath(inst.path))})',
                           xaxis=dict(title='time(sec)'),
                           yaxis=dict(title='Y-axis'))
        fig = go.Figure(data=data, layout=layout)
        iplot(fig)

In [ ]:
# ToDo: improve code duplication situation
# (can't fully integrate this in the above loop though, because some contact time-windows are removed there, thus need to iterate twice)

X, y = [], []

for inst in instances:
    contact_time, contact_time_index = get_next_contact_time(inst.df, -1)

    last_contact_end_time = reference_end_time
    while True:
        contact_time, contact_time_index = get_next_contact_time(
            inst.df, last_contact_end_time)
        if contact_time is None:
            break

        # append contact time-window to feature matrix
        # to equalize time-window size per instance, all time-windows are created by taking inst.window_size many elements (rows) from the first element of a contact time-window
        X.append(
            inst.df.iloc[contact_time_index:contact_time_index+inst.window_size][target].to_numpy())
        y.append(inst.contact_type)

        _, _, contact_end_time = get_contact_duration(inst.df, contact_time)
        last_contact_end_time = contact_end_time

# pad smaller windows with arrays of zeros to equalize window lengths across instances
max_window_size = max([len(x) for x in X])
for i in range(len(X)):
    X[i] = np.vstack([X[i], np.zeros(
        (max_window_size - len(X[i]), len(target)))])

X = np.array(X)
print(np.shape(X))
print(np.shape(y))